In [ ]:
from IPython.display import display  # We will display() explicitly
import serialhub  # Import our plugin package, must install if this fails
print(f"SerialHub python package version: {serialhub.__version__}")

W = serialhub.SerialHubWidget()  # Instantiate our custom widget
# IMPORTANT: Match settings to your device
W.serial_options = {'baudRate': 115200}
display(W)  # Render the widget JavaScript/DOM component visually in cell
#Could "return" W variable from cell to render without using display()

print(f"Premature check if supported: {W.is_supported}")  # Expect "False"
"If the widgets button indicates 'Supported' then CLICK IT!!!"

In [ ]:
print(f"Supported: {W.is_supported}") #Check AFTER the widget frontend syncs up
#If reports False, you likely need Google Chrome browser for WebSerial API
W.value = "CLEARED" #Show something in widget debug/output area
to_send = f"Test {W.status}\n" #Read the status-button text into a backend string
W.write_str(to_send) #Send a string to the serial port, hopefully statistics increase
to_send #Show what we tried to send

In [ ]:
backend_stats = [W.pkt_send_back, W.pkt_recv_back] #Capture backend stats traitlets
frontend_stats = [W.pkt_send_front, W.pkt_recv_front] #Capture frontend stats traitlets
print("Statistics are each a Tuple(bytes, packets):") #Note that stats are a PAIR of ints
print(f"Backend  [send, recv] {backend_stats}") #Hopefully the above cell sent data via backend
print(f"Frontend [send, recv] {frontend_stats}") #Hopefully the above cell sent data via backend
frontend_stats == backend_stats #Did frontend javascript send/recv matching amount???

In [ ]:
#Try something else here, like writing more data directly to the widget
W.write_bytes(b'Test\x0D\x0A') #Send some "binary" data (the "native" data type)
W.pkt_send_front #Front-end stats need some time to sync up

Now we try reading some data via callback mechanism...

In [ ]:
CNT=0 #GLOBAL variable as a simple counter
def gotbuf_cb(buf): #Create a callback function
    global CNT #Access existing global var, not a new local one
    CNT = CNT + 1 #Increment "packet" counter
    W.value = f"Value now {CNT}" #Show message in widget debug area

W.on_recv(gotbuf_cb) #Install our custom callback

In [ ]:
CNT #Hopefully our counter goes up if you receive any serial data

In [ ]:
W.on_recv(None) #Uninstall our callback
CNT #Our callback should no longer be called

Now we start testing out the SerialIO wrapper which mimicks traditional IO read/write

In [ ]:
SER = serialhub.SerialIO(W) #Start testing new "SerialIO" helper/wrapper
SER.closed() #Expect False, especially if SerialHubPort is active

In [ ]:
#SER.write(b'TEST') #Widget write stats should go up

In [ ]:
SER.in_waiting #IF your serial device has written recently, should have accumulated some data

In [ ]:
#SER.readall() #This should drain any accumulated data so far